In [1]:
from tabulate import tabulate  # Para exibir resultados formatados

# Função para arredondar e verificar duplicidade
def obter_localizacoes(nlong):
    poslong = []
    for i in range(1, nlong + 1):
        while True:
            try:
                localizacao = round(float(input(f"Digite a localização da {i}ª longarina (em metros): ")), 2)
                if localizacao < 0:
                    print("Erro: A localização não pode ser negativa.")
                elif localizacao in poslong:
                    print("Erro: Já existe uma longarina nessa posição.")
                else:
                    poslong.append(localizacao)
                    break
            except ValueError:
                print("Por favor, insira um valor numérico válido.")
    poslong.sort()  # Ordena as localizações das longarinas
    return poslong

# Função para calcular as localizações relativas das longarinas
def calcular_relativas(localizacoes, eixo):
    relativas = [round((loc - eixo if loc >= eixo else eixo - loc) * (-1 if loc < eixo else 1), 2) for loc in localizacoes]
    
    return relativas

# Função para calcular comprimento
def calcular_comprimento(poslong):
    if len(poslong) == 1:
        return round(2 * poslong[0], 2)
    return round((poslong[-1] - poslong[0]) + (2 * poslong[0]), 2)

# Função para calcular as cargas unitárias
def cargas(localizacoes, comprimento, distguardpist, compguardcorp):
    # Verifica se a localização da primeira longarina é menor ou igual a distância de início da pista
    if localizacoes[0] == distguardpist + compguardcorp:
        cargas_units = [0, compguardcorp] + localizacoes + [comprimento - compguardcorp, comprimento]
    elif localizacoes[0] == compguardcorp:
        cargas_units = [0, distguardpist + compguardcorp] + localizacoes + [comprimento - (distguardpist + compguardcorp), comprimento]
    else:
        cargas_units = [0, compguardcorp, distguardpist + compguardcorp] + localizacoes + [comprimento - (compguardcorp + distguardpist), comprimento - compguardcorp, comprimento]
    
    return sorted([round(c, 2) for c in cargas_units])  # Ordena as localizações das cargas

# Função para calcular os valores de rij
def calcular_rij(nlong, poslong, cargas_relativas, disteixo):
    vigas_relativas = [round(viga - disteixo, 2) for viga in poslong]
    somatorio_quadrados = sum([viga ** 2 for viga in vigas_relativas])

    resultados_rij = []
    for i, viga in enumerate(vigas_relativas):
        if somatorio_quadrados == 0:  # Verifica se o somatório dos quadrados é zero
            rij_por_longarina = [round(1 / nlong, 4) for _ in cargas_relativas]
        else:
            rij_por_longarina = [
                round((1 / nlong) + ((viga * carga) / somatorio_quadrados), 4) for carga in cargas_relativas
            ]
        resultados_rij.append({"longarina": round(poslong[i], 2), "resultados": rij_por_longarina})
    return resultados_rij

# Função para exibir os resultados
def exibir_rij(resultados_rij, cargas_units):
    headers = ["Longarina"] + [f"Carga {i+1} ({pos:.2f}m)" for i, pos in enumerate(cargas_units)]
    tabela = [
        [f"{res['longarina']:.2f}m"] + [f"{rij:.4f}" for rij in res["resultados"]]
        for res in resultados_rij
    ]
    print("\nResultados de rij (formatados):")
    print(tabulate(tabela, headers=headers, tablefmt="fancy_grid"))

# Função principal
def main():
    # Pergunta o comprimento da ponte
    while True:
        try:
            tamanhopont = float(input("Qual o comprimento da sua ponte? "))
            if tamanhopont < 0:
                print("Por favor, insira um número maior que 0.")
            else:
                break
        except ValueError:
            print("Por favor, insira um número válido.")

     # Pergunta o número de longarinas
    while True:
        try:
            nlong = int(input("Quantas longarinas você quer calcular? "))
            if nlong < 1:
                print("Por favor, insira um número maior ou igual a 1.")
            else:
                break
        except ValueError:
            print("Por favor, insira um número inteiro válido.")

    # Pergunta a rigidez da longarina
    while True:
        try:
            rigidlong = float(input("Qual a rigidez média da(s) sua(s) longarina(s)?"))
            if rigidlong < 0:
                print("Por favor, insira um número maior ou igual a 0.")
            else:
                break
        except ValueError:
            print("Por favor, insira um número inteiro válido.")

     # Pergunta o número de transversinas
    while True:
        try:
            ntransv = int(input("Quantas transversinas possuem na sua ponte?"))
            if ntransv < 1:
                print("Por favor, insira um número maior ou igual a 1.")
            else:
                break
        except ValueError:
            print("Por favor, insira um número inteiro válido.")

    # Pergunta a rigidez da transversina
    while True:
        try:
            rigidtransv = float(input("Qual a rigidez média da(s) sua(s) transversina(s)?"))
            if rigidtransv < 0:
                print("Por favor, insira um número maior ou igual a 0.")
            else:
                break
        except ValueError:
            print("Por favor, insira um número inteiro válido.")

    # Pergunta o comprimento do guarda-corpo
    while True:
        try:
            compguardcorp = float(input("Qual o comprimento do guarda-corpo? "))
            if compguardcorp < 0:
                print("Por favor, insira um número maior ou igual a 0.")
            else:
                break
        except ValueError:
            print("Por favor, insira um número válido.")

    # Pergunta a distância entre o guarda-corpo e a pista
    while True:
        try:
            distguardpist = float(input("Qual a distância entre o guarda-corpo e o início da pista? "))
            if distguardpist < 0:
                print("Por favor, insira um número maior ou igual a 0.")
            else:
                break
        except ValueError:
            print("Por favor, insira um número válido.")

    # Obtém as localizações das longarinas
    localizacoes = obter_localizacoes(nlong)
    print("\nLocalizações das longarinas (ordenadas):", localizacoes)

    # Calcula o comprimento da seção transversal
    comprimento = calcular_comprimento(localizacoes)

    # Calculo de lambda
    y = (comprimento / 2 * tamanhopont) * ((tamanhopont * nlong * rigidlong) / ((comprimento * ntransv * rigidtransv) ** (1/4))) 
    if y <= 0.3:
        print("\nSua ponte pode ser calculada pelo método de engesser-courbon!")
    else:
        print("\nSua ponte não é adequada para a utilização do método de engesser-courbon!")

    # Mostra o fator lambda, comprimento e eixo
    print("Seu fator lambda é:", round(y, 2))
    print("\nO comprimento total da seção transversal é:", comprimento, "metros")
    disteixo = comprimento / 2
    print(f"O eixo (meio da seção transversal) é: {round(disteixo, 2)} metros")

    # Calcula as localizações relativas das longarinas
    vigas_relativas = calcular_relativas(localizacoes, disteixo)
    print("\nLocalizações das longarinas em relação ao eixo:")
    for i, viga in enumerate(vigas_relativas, 1):
        print(f"Longarina {i}: {viga} metros")

    # Calcula as cargas unitárias e suas localizações relativas
    cargas_units = cargas(localizacoes, comprimento, distguardpist, compguardcorp)
    print("\nLocalização das cargas unitárias:", cargas_units)
    cargas_relativas = calcular_relativas(cargas_units, disteixo)
    print("\nLocalizações das cargas em relação ao eixo:")
    for i, carga in enumerate(cargas_relativas, 1):
        print(f"Carga {i}: {carga} metros")

    # Calcula os valores de rij
    resultados_rij = calcular_rij(nlong, localizacoes, cargas_relativas, disteixo)

    # Exibe os resultados de rij
    exibir_rij(resultados_rij, cargas_units)

    # Cálculo da linha neutra com o comprimento do eixo
    if nlong > 1:  # Só calcula a linha neutra se houver mais de uma longarina
        somatorio_quadrados = sum([viga ** 2 for viga in vigas_relativas])
        linha_neutra = (1 / nlong) * somatorio_quadrados / abs(vigas_relativas[0]) + disteixo  # Adiciona o eixo
        print(f"\nLocalização da linha neutra: {round(linha_neutra, 2)} metros")
    else:
        print("\nA longarina absorve toda a carga da seção transversal")

    # Cálculo de q1, q2 e P
    somatorio_quadrados = sum([v ** 2 for v in vigas_relativas])
    if nlong > 1:
        roda1 = round((1 / nlong) + ((vigas_relativas[0] * (distguardpist+0.5+compguardcorp-disteixo)) / somatorio_quadrados), 4)
        roda2 = round((1 / nlong) + ((vigas_relativas[0] * (distguardpist+compguardcorp+2.5-disteixo)) / somatorio_quadrados), 4)
    else:
        roda1 = 1
        roda2 = 1

    P = 75 * roda1 + 75 * roda2
    print(f"\nCarga (P): {round(P, 2)} KN")

    if nlong > 1:
        rijq1 = (1 / nlong) + ((vigas_relativas[0] * (- disteixo + compguardcorp)) / somatorio_quadrados)  # Rij para a primeira longarina e carga no guarda-corpo
        rijq2 = (1 / nlong) + ((vigas_relativas[0] * (- disteixo + compguardcorp + 3)) / somatorio_quadrados)
    
        q1 = round(5 * rijq1 * (linha_neutra - compguardcorp) / 2, 2)
        q2 = round(5 * rijq2 * (linha_neutra - compguardcorp - 3) / 2, 2)
    else:
        q1 = round(5 * (comprimento - (2 * compguardcorp)), 2)
        q2 = round(5 * (comprimento - (2 * compguardcorp) - 3), 2)

    print(f"Valor calculado de q1: {q1} KN/m")
    print(f"Valor calculado de q2: {q2} KN/m")

    # Calcula o coeficiente de impacto vertical
    if tamanhopont < 10:
        civ = 1.35
    else: 
        civ = 1 + 1.06 * (20 / (tamanhopont + 50))

    # Calcula o coeficiente de número de faixas
    while True:
        try:
            numfaixas = int(input("Quantas faixas possuem na sua ponte?"))
            if numfaixas < 2:
                print("Por favor, insira um número igual ou maior que 2.")
            else:
                break
        except ValueError:
            print("Por favor, insira um número válido.")

    cnf = 1 - 0.05 * (numfaixas - 2)

    # Calcula o coeficiente de impacto adicional 
    while True:
        try:
            material = str(input("De qual material é feita sua ponte?"))
            if material == "concreto" or material == "mista":
                cia = 1.25
                break
            elif material == "aço":
                cia = 1.15
                break
            else:
                print("Material inválido. Por favor, insira 'concreto', 'mista' ou 'aço'.")
        except Exception as e:
            print(f"Ocorreu um erro: {e}. Tente novamente.")

    # #Calcula o fator de impacto
    fatorimpact = civ * cnf * cia 
    print(f"\nSeu fator de impacto é: {round(fatorimpact, 2)}")

    # Mostra as cargas majoradas
    Pmaj = P * fatorimpact
    q1maj = q1 * fatorimpact
    q2maj = q2 * fatorimpact


    print(f"\nValor calculado de P majorado: {round(Pmaj, 2)} KN")
    print(f"Valor calculado de q1 majorado: {round(q1maj, 2)} KN/m")
    print(f"Valor calculado de q2 majorado: {round(q2maj,2)} KN/m")

# Chama a função principal
main()


Localizações das longarinas (ordenadas): [0.6, 3.1, 5.6, 8.1, 10.6, 13.1, 15.6]

Sua ponte não é adequada para a utilização do método de engesser-courbon!
Seu fator lambda é: 101743.53

O comprimento total da seção transversal é: 16.2 metros
O eixo (meio da seção transversal) é: 8.1 metros

Localizações das longarinas em relação ao eixo:
Longarina 1: -7.5 metros
Longarina 2: -5.0 metros
Longarina 3: -2.5 metros
Longarina 4: 0.0 metros
Longarina 5: 2.5 metros
Longarina 6: 5.0 metros
Longarina 7: 7.5 metros

Localização das cargas unitárias: [0, 0.4, 0.4, 0.6, 3.1, 5.6, 8.1, 10.6, 13.1, 15.6, 15.8, 15.8, 16.2]

Localizações das cargas em relação ao eixo:
Carga 1: -8.1 metros
Carga 2: -7.7 metros
Carga 3: -7.7 metros
Carga 4: -7.5 metros
Carga 5: -5.0 metros
Carga 6: -2.5 metros
Carga 7: 0.0 metros
Carga 8: 2.5 metros
Carga 9: 5.0 metros
Carga 10: 7.5 metros
Carga 11: 7.7 metros
Carga 12: 7.7 metros
Carga 13: 8.1 metros

Resultados de rij (formatados):
╒═════════════╤═══════════════════╤